<a href="https://colab.research.google.com/github/mattambrogi/llama-index-experiments/blob/main/copy_base_index_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Scrape collection of blog posts
# Grab date and append to title
# Place in directory 'blog_posts/' Put inside multiple folders.
# Implementation details hidden

offset = 0
increment = 20
total = 1000
while True:
  scraped_posts = GetPosts({'number': increment, 'offset': offset})
  if len(scraped_posts) == 0:
    break  # no more posts returned

  for post in scraped_posts:
    transformed_title = re.sub(r'[\s/]+', '_', post.title)
    with open('blog_posts/' + transformed_title + '.txt', 'w') as f:
      post.content = result = re.sub(r'<.*?>', '', post.content)
      f.write(post.content)

  offset = offset + increment
  if offset >= total:
    break

In [ ]:
!pip install openai
!pip install llama_index
!pip install langchain

from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader, LLMPredictor, PromptHelper, ServiceContext
from langchain.chat_models import ChatOpenAI

In [ ]:
os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"

In [ ]:
documents = SimpleDirectoryReader('blog_posts').load_data()

In [ ]:
max_input_size = 4096
num_outputs = 256
chunk_size_limit = 600

llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))
prompt_helper = PromptHelper(max_input_size, num_outputs, chunk_size_limit=chunk_size_limit)
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper = prompt_helper)

In [ ]:
index = GPTVectorStoreIndex.from_documents(
    documents, service_context=service_context
)

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=3)

In [ ]:
response = query_engine.query("Example question")
print(response)